In [1]:
import pandas as pd
from campitos import campos
from create_tables import *
import csv
import json

def csv_to_fixture(csv_path, model_name, pk_start=1):
    with open(csv_path, 'r') as f:
        reader = csv.DictReader(f)
        fixtures = []
        pk = pk_start
        for row in reader:
            # Convertir PK a cadena para la asignación
            fixture = {
                'model': model_name,
                'pk': str(pk),
                'fields': {}
            }
            for field, value in row.items():
                if field != 'pk':  # Asegurar que no sobreescribimos el PK
                    fixture['fields'][field] = value
            fixtures.append(fixture)
            pk += 1  # Incrementar PK para el próximo registro
    return fixtures




def merge_fixtures(fixtures, output_path):
    """
    Merge a list of fixture dictionaries into a single fixture file.

    Args:
        fixtures (list): A list of fixture dictionaries.
        output_path (str or Path): The path to the output file.

    Returns:
        None
    """
    merged = []
    for fixture in fixtures:
        merged.extend(fixture)
    with open(output_path, 'w') as f:
        json.dump(merged, f, indent=4)


def horizon_table(gsid, zone_id):
    horizon_columns = {
        'equipo': 'pump', 
        'sector': zone_id, 
        'horizonte':'h_level', 
        'z_real':'z_real', 
        'cc_real':'cc_real', 
        'pmp_real':'pmp_real',
       'pedregosidad':'pedregosity', 
        }  
    f = spreadsheet_to_pandas('info_horizontes',gsid)
    f = f.rename(columns=horizon_columns)
    return f




for i, field in enumerate(campos):
    print(f'working on {field}')
    id_db = campos[field]['id']
    id = campos[field]['spreadsheet_id']
    if i == 0:
        fields = fields_table(id, id_db)
        z, p = zone_and_pump_tables(id, id_db)
        h = horizon_table(id, id_db)
    else:
        fields2 = fields_table(id, id_db)
        fields = pd.concat([fields,fields2])
        z2, p2 = zone_and_pump_tables(id, id_db)
        z = pd.concat([z, z2])
        p = pd.concat([p, p2])
        h2 = horizon_table(id, id_db)
        h = pd.concat([h, h2])



fields.to_csv('./csvs/fields.csv', index=False)
p.to_csv('./csvs/p.csv', index=False)
z.to_csv('./csvs/z.csv', index=False)
h.to_csv('./csvs/h.csv', index=False)

working on 23Bellavista1
working on 23Bellavista2
working on LasNueces
working on LosMaitenes


In [5]:
zone_pk_map = {}  # Diccionario para rastrear los PKs de las zones

# Procesamiento de zones y creación del mapa de PKs
for i, row in z.iterrows():
    unique_identifier = row['dropcontrol_id']  # Asume que hay un identificador único
    zone_pk_map[unique_identifier] = str(i + 1)  # Asume que el PK es simplemente el índice + 1

In [6]:
zone_pk_map

{26735: '1',
 26736: '2',
 26737: '3',
 26738: '4',
 22981: '5',
 22982: '6',
 22983: '7',
 22984: '8',
 26739: '9',
 26740: '10',
 26741: '11',
 26742: '12',
 45623: '13',
 45624: '14',
 45625: '15',
 33720: '16',
 33721: '17',
 33722: '18',
 33723: '19',
 33640: '20',
 33641: '21',
 33642: '22',
 33643: '23',
 22977: '24',
 22978: '25',
 22979: '26',
 22980: '27',
 22973: '28',
 22974: '29',
 22975: '30',
 22976: '31',
 26743: '32',
 26744: '33',
 26745: '34',
 26746: '35',
 26747: '36',
 48420: '1',
 48421: '2',
 48422: '3',
 48423: '4',
 39269: '5',
 39270: '6',
 39271: '7',
 39272: '8',
 39273: '9',
 39274: '10',
 39275: '11',
 39276: '12',
 39285: '13',
 39286: '14',
 39287: '15',
 39288: '16',
 39277: '17',
 39278: '18',
 39279: '19',
 39280: '20',
 39281: '21',
 39282: '22',
 39283: '23',
 39284: '24',
 3863: '1',
 3864: '2',
 3865: '3',
 3866: '4',
 3868: '5',
 3869: '6',
 3870: '7',
 3871: '8',
 3872: '9',
 3522: '1',
 3523: '2',
 3524: '3',
 3525: '4',
 3526: '5',
 3527: '6'

In [2]:
fixtures = [
    csv_to_fixture('./csvs/fields.csv', 'gota_app.seasonfield'),
    csv_to_fixture('./csvs/p.csv', 'gota_app.pump'),
    csv_to_fixture('./csvs/z.csv', 'gota_app.zone'),
    #csv_to_fixture('./csvs/h.csv', 'gota_app.Horizon', 'id'),
]

In [3]:
merge_fixtures(fixtures, 'db_content.json')